In [43]:
class DataParticleValue():
    JSON_DATA = "JSON_Data"
    ENG = "eng"
    OK = "ok"
    CHECKSUM_FAILED = "checksum_failed"
    OUT_OF_RANGE = "out_of_range"
    INVALID = "invalid"
    QUESTIONABLE = "questionable"

In [44]:
class DataParticle(object):
    """
    This class is responsible for storing and ultimately generating data
    particles in the designated format from the associated inputs. It
    fills in fields as necessary, and is a valid Data Particle
    that can be sent up to the InstrumentAgent.

    It is the intent that this class is subclassed as needed if an instrument must
    modify fields in the outgoing packet. The hope is to have most of the superclass
    code be called by the child class with just values overridden as needed.
    """

    # data particle type is intended to be defined in each derived data particle class.  This value should be unique
    # for all data particles.  Best practice is to access this variable using the accessor method:
    # data_particle_type()
    _data_particle_type = None

    def __init__(self, raw_data,
                 port_timestamp=None,
                 internal_timestamp=None,
                 preferred_timestamp=DataParticleKey.PORT_TIMESTAMP,
                 quality_flag=DataParticleValue.OK,
                 new_sequence=None):
        """ Build a particle seeded with appropriate information

        @param raw_data The raw data used in the particle
        """
        if new_sequence is not None and not isinstance(new_sequence, bool):
            raise TypeError("new_sequence is not a bool")

        self.contents = {
            DataParticleKey.PKT_FORMAT_ID: DataParticleValue.JSON_DATA,
            DataParticleKey.PKT_VERSION: 1,
            DataParticleKey.PORT_TIMESTAMP: port_timestamp,
            DataParticleKey.INTERNAL_TIMESTAMP: internal_timestamp,
            DataParticleKey.DRIVER_TIMESTAMP: ntplib.system_to_ntp_time(time.time()),
            DataParticleKey.PREFERRED_TIMESTAMP: preferred_timestamp,
            DataParticleKey.QUALITY_FLAG: quality_flag,
        }
        self._encoding_errors = []
        if new_sequence is not None:
            self.contents[DataParticleKey.NEW_SEQUENCE] = new_sequence

        self.raw_data = raw_data

    def __eq__(self, arg):
        """
        Equality check for testing purposes.
        """
        if self.data_particle_type() != arg.data_particle_type():
            log.debug('Data particle type does not match: %s %s', self.data_particle_type(), arg.data_particle_type())
            return False

        if self.raw_data != arg.raw_data:
            log.debug('Raw data does not match')
            return False

        generated1 = self.generate()
        generated2 = arg.generate()
        missing, differing = self._compare(generated1, generated2, ignore_keys=[DataParticleKey.DRIVER_TIMESTAMP,
                                                                                DataParticleKey.PREFERRED_TIMESTAMP])
        if missing:
            log.error('Key mismatch between particle dictionaries: %r', missing)
            return False

        if differing:
            log.error('Value mismatch between particle dictionaries: %r', differing)

        return True

    @staticmethod
    def _compare(d1, d2, ignore_keys=None):
        ignore_keys = ignore_keys if ignore_keys else []
        missing = set(d1).symmetric_difference(d2)
        differing = {}
        for k in d1:
            if k in ignore_keys or k in missing:
                continue
            if d1[k] != d2[k]:
                differing[k] = (d1[k], d2[k])

        return missing, differing

    @classmethod
    def type(cls):
        """
        return the data particle type
        @return: data particle type
        """
        return cls._data_particle_type

    def set_internal_timestamp(self, timestamp=None, unix_time=None):
        """
        Set the internal timestamp
        @param timestamp: NTP timestamp to set
        @param unix_time: Unix time as returned from time.time()
        @raise InstrumentParameterException if timestamp or unix_time not supplied
        """
        if timestamp is None and unix_time is None:
            raise InstrumentParameterException("timestamp or unix_time required")

        if unix_time is not None:
            timestamp = ntplib.system_to_ntp_time(unix_time)

        self.contents[DataParticleKey.INTERNAL_TIMESTAMP] = float(timestamp)

    def set_value(self, value_id, value):
        """
        Set a content value, restricted as necessary

        @param value_id The ID of the value to set, should be from DataParticleKey
        @param value The value to set
        @raises ReadOnlyException If the parameter cannot be set
        """
        if (value_id == DataParticleKey.INTERNAL_TIMESTAMP) and (self._check_timestamp(value)):
            self.contents[DataParticleKey.INTERNAL_TIMESTAMP] = value
        else:
            raise ReadOnlyException("Parameter %s not able to be set to %s after object creation!" %
                                    (value_id, value))

    def get_value(self, value_id):
        """ Return a stored value

        @param value_id The ID (from DataParticleKey) for the parameter to return
        @raises NotImplementedException If there is an invalid id
        """
        if DataParticleKey.has(value_id):
            return self.contents[value_id]
        else:
            raise NotImplementedException("Value %s not available in particle!", value_id)

    def data_particle_type(self):
        """
        Return the data particle type (aka stream name)
        @raise: NotImplementedException if _data_particle_type is not set
        """
        if self._data_particle_type is None:
            raise NotImplementedException("_data_particle_type not initialized")

        return self._data_particle_type

    def generate_dict(self):
        """
        Generate a simple dictionary of sensor data and timestamps, without
        going to JSON. This is useful for the times when JSON is not needed to
        go across an interface. There are times when particles are used
        internally to a component/process/module/etc.
        @retval A python dictionary with the proper timestamps and data values
        @throws InstrumentDriverException if there is a problem wtih the inputs
        """
        # Do we wan't downstream processes to check this?
        # for time in [DataParticleKey.INTERNAL_TIMESTAMP,
        #             DataParticleKey.DRIVER_TIMESTAMP,
        #             DataParticleKey.PORT_TIMESTAMP]:
        #    if  not self._check_timestamp(self.contents[time]):
        #        raise SampleException("Invalid port agent timestamp in raw packet")

        # verify preferred timestamp exists in the structure...
        if not self._check_preferred_timestamps():
            raise SampleException("Preferred timestamp not in particle!")

        # build response structure
        self._encoding_errors = []
        values = self._build_parsed_values()

        if all([self.contents[DataParticleKey.PREFERRED_TIMESTAMP] == DataParticleKey.PORT_TIMESTAMP,
                self.contents[DataParticleKey.PORT_TIMESTAMP] == 0,
                self.contents[DataParticleKey.INTERNAL_TIMESTAMP] is not None]):
            self.contents[DataParticleKey.PREFERRED_TIMESTAMP] = DataParticleKey.INTERNAL_TIMESTAMP

        result = self._build_base_structure()
        result[DataParticleKey.STREAM_NAME] = self.data_particle_type()
        result[DataParticleKey.VALUES] = values
        return result

    def generate(self, sorted=False):
        """
        Generates a JSON_parsed packet from a sample dictionary of sensor data and
        associates a timestamp with it

        @param sorted ignored, maintained only to avoid breaking drivers
        @return A dictionary representing this particle
        @throws InstrumentDriverException If there is a problem with the inputs
        """
        return self.generate_dict()

    def _build_parsed_values(self):
        """
        Build values of a parsed structure. Just the values are built so
        so that a child class can override this class, but call it with
        super() to get the base structure before modification

        @return the values tag for this data structure ready to JSONify
        @raises SampleException when parsed values can not be properly returned
        """
        raise SampleException("Parsed values block not overridden")

    def _build_base_structure(self):
        """
        Build the base/header information for an output structure.
        Follow on methods can then modify it by adding or editing values.

        @return A fresh copy of a core structure to be exported
        """
        result = dict(self.contents)
        # clean out optional fields that were missing
        if not self.contents[DataParticleKey.PORT_TIMESTAMP]:
            del result[DataParticleKey.PORT_TIMESTAMP]
        if not self.contents[DataParticleKey.INTERNAL_TIMESTAMP]:
            del result[DataParticleKey.INTERNAL_TIMESTAMP]
        return result

    def _check_timestamp(self, timestamp):
        """
        Check to make sure the timestamp is reasonable

        @param timestamp An NTP4 formatted timestamp (64bit)
        @return True if timestamp is okay or None, False otherwise
        """
        if timestamp is None:
            return True
        if not isinstance(timestamp, float):
            return False

        # is it sufficiently in the future to be unreasonable?
        if timestamp > ntplib.system_to_ntp_time(time.time() + (86400 * 365)):
            return False
        else:
            return True

    def _check_preferred_timestamps(self):
        """
        Check to make sure the preferred timestamp indicated in the
        particle is actually listed, possibly adjusting to 2nd best
        if not there.

        @throws SampleException When there is a problem with the preferred
            timestamp in the sample.
        """
        if self.contents[DataParticleKey.PREFERRED_TIMESTAMP] is None:
            raise SampleException("Missing preferred timestamp, %s, in particle" %
                                  self.contents[DataParticleKey.PREFERRED_TIMESTAMP])

        # This should be handled downstream.  Don't want to not publish data because
        # the port agent stopped putting out timestamps
        # if self.contents[self.contents[DataParticleKey.PREFERRED_TIMESTAMP]] == None:
        #    raise SampleException("Preferred timestamp, %s, is not defined" %
        #                          self.contents[DataParticleKey.PREFERRED_TIMESTAMP])

        return True

    def _encode_value(self, name, value, encoding_function):
        """
        Encode a value using the encoding function, if it fails store the error in a queue
        """
        encoded_val = None

        try:
            encoded_val = encoding_function(value)
        except Exception:
            log.error("Data particle error encoding. Name:%s Value:%s", name, value)
            self._encoding_errors.append({name: value})
        return {DataParticleKey.VALUE_ID: name,
                DataParticleKey.VALUE: encoded_val}

    def get_encoding_errors(self):
        """
        Return the encoding errors list
        """
        return self._encoding_errors


In [47]:
# %load data_particle.py
#!/usr/bin/env python

"""
@package mi.core.instrument.data_particle_generator Base data particle generator
@file mi/core/instrument/data_particle_generator.py
@author Steve Foley
@brief Contains logic to generate data particles to be exchanged between
the driver and agent. This involves a JSON interchange format
"""
import json
import time
import ntplib
import base64

from mi.core.common import BaseEnum
from mi.core.exceptions import SampleException, ReadOnlyException, NotImplementedException, InstrumentParameterException
from mi.core.log import get_logger


__author__ = 'Steve Foley'
__license__ = 'Apache 2.0'

log = get_logger()


class CommonDataParticleType(BaseEnum):
    """
    This enum defines all the common particle types defined in the modules.  Currently there is only one, but by
    using an enum here we have the opportunity to define more common data particles.
    """
    RAW = "raw"


class DataParticleKey(BaseEnum):
    PKT_FORMAT_ID = "pkt_format_id"
    PKT_VERSION = "pkt_version"
    STREAM_NAME = "stream_name"
    INTERNAL_TIMESTAMP = "internal_timestamp"
    PORT_TIMESTAMP = "port_timestamp"
    DRIVER_TIMESTAMP = "driver_timestamp"
    PREFERRED_TIMESTAMP = "preferred_timestamp"
    QUALITY_FLAG = "quality_flag"
    VALUES = "values"
    VALUE_ID = "value_id"
    VALUE = "value"
    BINARY = "binary"
    NEW_SEQUENCE = "new_sequence"


class DataParticleValue(BaseEnum):
    JSON_DATA = "JSON_Data"
    ENG = "eng"
    OK = "ok"
    CHECKSUM_FAILED = "checksum_failed"
    OUT_OF_RANGE = "out_of_range"
    INVALID = "invalid"
    QUESTIONABLE = "questionable"


class DataParticle(object):
    """
    This class is responsible for storing and ultimately generating data
    particles in the designated format from the associated inputs. It
    fills in fields as necessary, and is a valid Data Particle
    that can be sent up to the InstrumentAgent.

    It is the intent that this class is subclassed as needed if an instrument must
    modify fields in the outgoing packet. The hope is to have most of the superclass
    code be called by the child class with just values overridden as needed.
    """

    # data particle type is intended to be defined in each derived data particle class.  This value should be unique
    # for all data particles.  Best practice is to access this variable using the accessor method:
    # data_particle_type()
    _data_particle_type = None

    def __init__(self, raw_data,
                 port_timestamp=None,
                 internal_timestamp=None,
                 preferred_timestamp=DataParticleKey.PORT_TIMESTAMP,
                 quality_flag=DataParticleValue.OK,
                 new_sequence=None):
        """ Build a particle seeded with appropriate information

        @param raw_data The raw data used in the particle
        """
        if new_sequence is not None and not isinstance(new_sequence, bool):
            raise TypeError("new_sequence is not a bool")

        self.contents = {
            DataParticleKey.PKT_FORMAT_ID: DataParticleValue.JSON_DATA,
            DataParticleKey.PKT_VERSION: 1,
            DataParticleKey.PORT_TIMESTAMP: port_timestamp,
            DataParticleKey.INTERNAL_TIMESTAMP: internal_timestamp,
            DataParticleKey.DRIVER_TIMESTAMP: ntplib.system_to_ntp_time(time.time()),
            DataParticleKey.PREFERRED_TIMESTAMP: preferred_timestamp,
            DataParticleKey.QUALITY_FLAG: quality_flag,
        }
        self._encoding_errors = []
        if new_sequence is not None:
            self.contents[DataParticleKey.NEW_SEQUENCE] = new_sequence

        self.raw_data = raw_data

    def __eq__(self, arg):
        """
        Equality check for testing purposes.
        """
        if self.data_particle_type() != arg.data_particle_type():
            log.debug('Data particle type does not match: %s %s', self.data_particle_type(), arg.data_particle_type())
            return False

        if self.raw_data != arg.raw_data:
            log.debug('Raw data does not match')
            return False

        generated1 = self.generate()
        generated2 = arg.generate()
        missing, differing = self._compare(generated1, generated2, ignore_keys=[DataParticleKey.DRIVER_TIMESTAMP,
                                                                                DataParticleKey.PREFERRED_TIMESTAMP])
        if missing:
            log.error('Key mismatch between particle dictionaries: %r', missing)
            return False

        if differing:
            log.error('Value mismatch between particle dictionaries: %r', differing)

        return True

    @staticmethod
    def _compare(d1, d2, ignore_keys=None):
        ignore_keys = ignore_keys if ignore_keys else []
        missing = set(d1).symmetric_difference(d2)
        differing = {}
        for k in d1:
            if k in ignore_keys or k in missing:
                continue
            if d1[k] != d2[k]:
                differing[k] = (d1[k], d2[k])

        return missing, differing

    @classmethod
    def type(cls):
        """
        return the data particle type
        @return: data particle type
        """
        return cls._data_particle_type

    def set_internal_timestamp(self, timestamp=None, unix_time=None):
        """
        Set the internal timestamp
        @param timestamp: NTP timestamp to set
        @param unix_time: Unix time as returned from time.time()
        @raise InstrumentParameterException if timestamp or unix_time not supplied
        """
        if timestamp is None and unix_time is None:
            raise InstrumentParameterException("timestamp or unix_time required")

        if unix_time is not None:
            timestamp = ntplib.system_to_ntp_time(unix_time)

        self.contents[DataParticleKey.INTERNAL_TIMESTAMP] = float(timestamp)

    def set_value(self, value_id, value):
        """
        Set a content value, restricted as necessary

        @param value_id The ID of the value to set, should be from DataParticleKey
        @param value The value to set
        @raises ReadOnlyException If the parameter cannot be set
        """
        if (value_id == DataParticleKey.INTERNAL_TIMESTAMP) and (self._check_timestamp(value)):
            self.contents[DataParticleKey.INTERNAL_TIMESTAMP] = value
        else:
            raise ReadOnlyException("Parameter %s not able to be set to %s after object creation!" %
                                    (value_id, value))

    def get_value(self, value_id):
        """ Return a stored value

        @param value_id The ID (from DataParticleKey) for the parameter to return
        @raises NotImplementedException If there is an invalid id
        """
        if DataParticleKey.has(value_id):
            return self.contents[value_id]
        else:
            raise NotImplementedException("Value %s not available in particle!", value_id)

    def data_particle_type(self):
        """
        Return the data particle type (aka stream name)
        @raise: NotImplementedException if _data_particle_type is not set
        """
        if self._data_particle_type is None:
            raise NotImplementedException("_data_particle_type not initialized")

        return self._data_particle_type

    def generate_dict(self):
        """
        Generate a simple dictionary of sensor data and timestamps, without
        going to JSON. This is useful for the times when JSON is not needed to
        go across an interface. There are times when particles are used
        internally to a component/process/module/etc.
        @retval A python dictionary with the proper timestamps and data values
        @throws InstrumentDriverException if there is a problem wtih the inputs
        """
        # Do we wan't downstream processes to check this?
        # for time in [DataParticleKey.INTERNAL_TIMESTAMP,
        #             DataParticleKey.DRIVER_TIMESTAMP,
        #             DataParticleKey.PORT_TIMESTAMP]:
        #    if  not self._check_timestamp(self.contents[time]):
        #        raise SampleException("Invalid port agent timestamp in raw packet")

        # verify preferred timestamp exists in the structure...
        if not self._check_preferred_timestamps():
            raise SampleException("Preferred timestamp not in particle!")

        # build response structure
        self._encoding_errors = []
        values = self._build_parsed_values()

        if all([self.contents[DataParticleKey.PREFERRED_TIMESTAMP] == DataParticleKey.PORT_TIMESTAMP,
                self.contents[DataParticleKey.PORT_TIMESTAMP] == 0,
                self.contents[DataParticleKey.INTERNAL_TIMESTAMP] is not None]):
            self.contents[DataParticleKey.PREFERRED_TIMESTAMP] = DataParticleKey.INTERNAL_TIMESTAMP

        result = self._build_base_structure()
        result[DataParticleKey.STREAM_NAME] = self.data_particle_type()
        result[DataParticleKey.VALUES] = values
        return result

    def generate(self, sorted=False):
        """
        Generates a JSON_parsed packet from a sample dictionary of sensor data and
        associates a timestamp with it

        @param sorted ignored, maintained only to avoid breaking drivers
        @return A dictionary representing this particle
        @throws InstrumentDriverException If there is a problem with the inputs
        """
        return self.generate_dict()

    def _build_parsed_values(self):
        """
        Build values of a parsed structure. Just the values are built so
        so that a child class can override this class, but call it with
        super() to get the base structure before modification

        @return the values tag for this data structure ready to JSONify
        @raises SampleException when parsed values can not be properly returned
        """
        raise SampleException("Parsed values block not overridden")

    def _build_base_structure(self):
        """
        Build the base/header information for an output structure.
        Follow on methods can then modify it by adding or editing values.

        @return A fresh copy of a core structure to be exported
        """
        result = dict(self.contents)
        # clean out optional fields that were missing
        if not self.contents[DataParticleKey.PORT_TIMESTAMP]:
            del result[DataParticleKey.PORT_TIMESTAMP]
        if not self.contents[DataParticleKey.INTERNAL_TIMESTAMP]:
            del result[DataParticleKey.INTERNAL_TIMESTAMP]
        return result

    def _check_timestamp(self, timestamp):
        """
        Check to make sure the timestamp is reasonable

        @param timestamp An NTP4 formatted timestamp (64bit)
        @return True if timestamp is okay or None, False otherwise
        """
        if timestamp is None:
            return True
        if not isinstance(timestamp, float):
            return False

        # is it sufficiently in the future to be unreasonable?
        if timestamp > ntplib.system_to_ntp_time(time.time() + (86400 * 365)):
            return False
        else:
            return True

    def _check_preferred_timestamps(self):
        """
        Check to make sure the preferred timestamp indicated in the
        particle is actually listed, possibly adjusting to 2nd best
        if not there.

        @throws SampleException When there is a problem with the preferred
            timestamp in the sample.
        """
        if self.contents[DataParticleKey.PREFERRED_TIMESTAMP] is None:
            raise SampleException("Missing preferred timestamp, %s, in particle" %
                                  self.contents[DataParticleKey.PREFERRED_TIMESTAMP])

        # This should be handled downstream.  Don't want to not publish data because
        # the port agent stopped putting out timestamps
        # if self.contents[self.contents[DataParticleKey.PREFERRED_TIMESTAMP]] == None:
        #    raise SampleException("Preferred timestamp, %s, is not defined" %
        #                          self.contents[DataParticleKey.PREFERRED_TIMESTAMP])

        return True

    def _encode_value(self, name, value, encoding_function):
        """
        Encode a value using the encoding function, if it fails store the error in a queue
        """
        encoded_val = None

        try:
            encoded_val = encoding_function(value)
        except Exception:
            log.error("Data particle error encoding. Name:%s Value:%s", name, value)
            self._encoding_errors.append({name: value})
        return {DataParticleKey.VALUE_ID: name,
                DataParticleKey.VALUE: encoded_val}

    def get_encoding_errors(self):
        """
        Return the encoding errors list
        """
        return self._encoding_errors


class RawDataParticleKey(BaseEnum):
    PAYLOAD = "raw"
    LENGTH = "length"
    TYPE = "type"
    CHECKSUM = "checksum"


class RawDataParticle(DataParticle):
    """
    This class a common data particle for generating data particles of raw
    data.

    It essentially is a translation of the port agent packet
    """
    _data_particle_type = CommonDataParticleType.RAW

    def _build_parsed_values(self):
        """
        Build a particle out of a port agent packet.
        @returns A list that is ready to be added to the "values" tag before
           the structure is JSONified
        """

        port_agent_packet = self.raw_data
        if not isinstance(port_agent_packet, dict):
            raise SampleException("raw data not a dictionary")

        for param in ["raw", "length", "type", "checksum"]:
            if param not in port_agent_packet:
                raise SampleException("raw data not a complete port agent packet. missing %s" % param)

        payload = None
        length = None
        ptype = None
        checksum = None

        # Attempt to convert values
        try:
            payload = base64.b64encode(port_agent_packet.get("raw"))
        except TypeError:
            pass

        try:
            length = int(port_agent_packet.get("length"))
        except TypeError:
            pass

        try:
            ptype = int(port_agent_packet.get("type"))
        except TypeError:
            pass

        try:
            checksum = int(port_agent_packet.get("checksum"))
        except TypeError:
            pass

        result = [{
            DataParticleKey.VALUE_ID: RawDataParticleKey.PAYLOAD,
            DataParticleKey.VALUE: payload,
            DataParticleKey.BINARY: True},
            {
                DataParticleKey.VALUE_ID: RawDataParticleKey.LENGTH,
                DataParticleKey.VALUE: length},
            {
                DataParticleKey.VALUE_ID: RawDataParticleKey.TYPE,
                DataParticleKey.VALUE: ptype},
            {
                DataParticleKey.VALUE_ID: RawDataParticleKey.CHECKSUM,
                DataParticleKey.VALUE: checksum},
        ]

        return result


ModuleNotFoundError: No module named 'yaml'

In [48]:
# %load zplsc_c_echogram.py
"""
@package mi.dataset.driver.zplsc_c
@file mi/dataset/driver/zplsc_c/zplsc_c_echogram.py
@author Craig Risien/Rene Gelinas
@brief ZPLSC Echogram generation for the ooicore

Release notes:

This class supports the generation of ZPLSC-C echograms.
"""

import numpy as np
import mi.dataset.driver.zplsc_c.zplsc_functions as zf

__author__ = 'Rene Gelinas'


class ZplscCParameters(object):
    # TODO: This class should be replaced by methods to get the CCs from the system.
    # Configuration Parameters
    Salinity = 32   # Salinity in psu
    Pressure = 150  # in dbars (~ depth of instrument in meters).
    Bins2Avg = 1    # number of range bins to average - 1 is no averaging


class ZplscCCalibrationCoefficients(object):
    # TODO: This class should be replaced by methods to get the CCs from the system.
    ka = 464.3636
    kb = 3000.0
    kc = 1.893
    A = 0.001466
    B = 0.0002388
    C = 0.000000100335

    TVR = []
    VTX = []
    BP = []
    EL = []
    DS = []

    # Freq 38kHz
    TVR.append(1.691999969482e2)
    VTX.append(1.533999938965e2)
    BP.append(8.609999902546e-3)
    EL.append(1.623000030518e2)
    DS.append(2.280000038445e-2)

    # Freq 125kHz
    TVR.append(1.668999938965e2)
    VTX.append(5.8e+01)
    BP.append(1.530999969691e-2)
    EL.append(1.376999969482e2)
    DS.append(2.280000038445e-2)

    # Freq 200kHz
    TVR.append(1.688999938965e2)
    VTX.append(9.619999694824e1)
    BP.append(1.530999969691e-2)
    EL.append(1.456000061035e2)
    DS.append(2.250000089407e-2)

    # Freq 455kHz
    TVR.append(1.696000061035e2)
    VTX.append(1.301000061035e2)
    BP.append(8.609999902546e-3)
    EL.append(1.491999969482e2)
    DS.append(2.300000004470e-2)


class ZPLSCCEchogram(object):
    def __init__(self):
        self.cc = ZplscCCalibrationCoefficients()
        self.params = ZplscCParameters()

    def compute_backscatter(self, profile_hdr, chan_data, sound_speed, depth_range, sea_absorb):
        """
        Compute the backscatter volumes values for one zplsc_c profile data record.
        This code was borrowed from ASL MatLab code that reads in zplsc-c raw data
        and performs calculations in order to compute the backscatter volume in db.

        :param profile_hdr: Raw profile header with metadata from the zplsc-c instrument.
        :param chan_data: Raw frequency data from the zplsc-c instrument.
        :param sound_speed: Speed of sound at based on speed of sound, pressure and salinity.
        :param depth_range: Range of the depth of the measurements
        :param sea_absorb: Seawater absorption coefficient for each frequency
        :return: sv: Volume backscatter in db
        """

        __N = []
        if self.params.Bins2Avg > 1:
            for chan in range(profile_hdr.num_channels):
                el = self.cc.EL[chan] - 2.5/self.cc.DS[chan] + np.array(chan_data[chan])/(26214*self.cc.DS[chan])
                power = 10**(el/10)

                # Perform bin averaging
                num_bins = len(chan_data[chan])/self.params.Bins2Avg
                pwr_avg = []
                for _bin in range(num_bins):
                    pwr_avg.append(np.mean(power[_bin*self.params.Bins2Avg:(_bin+1)*self.params.Bins2Avg]))

                el_avg = 10*np.log10(pwr_avg)
                __N.append(np.round(26214*self.cc.DS[chan]*(el_avg - self.cc.EL[chan] + 2.5/self.cc.DS[chan])))

        else:
            for chan in range(profile_hdr.num_channels):
                __N.append(np.array(chan_data[chan]))

        sv = []
        for chan in range(profile_hdr.num_channels):
            # Calculate correction to Sv due to non square transmit pulse
            sv_offset = zf.compute_sv_offset(profile_hdr.frequency[chan], profile_hdr.pulse_length[chan])
            sv.append(self.cc.EL[chan]-2.5/self.cc.DS[chan] + __N[chan]/(26214*self.cc.DS[chan]) - self.cc.TVR[chan] -
                      20*np.log10(self.cc.VTX[chan]) + 20*np.log10(depth_range[chan]) +
                      2*sea_absorb[chan]*depth_range[chan] -
                      10*np.log10(0.5*sound_speed*profile_hdr.pulse_length[chan]/1e6*self.cc.BP[chan]) +
                      sv_offset)

        return sv

    def compute_echogram_metadata(self, profile_hdr):
        """
        Compute the metadata parameters needed to compute the zplsc-c volume backscatter values.

        :param  profile_hdr: Raw profile header with metadata from the zplsc-c instrument.
        :return: sound_speed : Speed of sound based on temperature, pressure and salinity.
                 depth_range : Range of depth values of the zplsc-c data.
                 sea_absorb : Sea absorption based on temperature, pressure, salinity and frequency.
        """

        # If the temperature sensor is available, compute the temperature from the counts.
        temperature = 0
        if profile_hdr.is_sensor_available:
            temperature = zf.zplsc_c_temperature(profile_hdr.temperature, self.cc.ka, self.cc.kb, self.cc.kc,
                                                 self.cc.A, self.cc.B, self.cc.C)

        sound_speed = zf.zplsc_c_ss(temperature, self.params.Pressure, self.params.Salinity)

        _m = []
        depth_range = []
        for chan in range(profile_hdr.num_channels):
            _m.append(np.array([x for x in range(1, (profile_hdr.num_bins[chan]/self.params.Bins2Avg)+1)]))
            depth_range.append(sound_speed*profile_hdr.lockout_index[0]/(2*profile_hdr.digitization_rate[0]) +
                               (sound_speed/4)*(((2*_m[chan]-1)*profile_hdr.range_samples[0]*self.params.Bins2Avg-1) /
                                                float(profile_hdr.digitization_rate[0]) +
                                                profile_hdr.pulse_length[0]/1e6))

        sea_absorb = []
        for chan in range(profile_hdr.num_channels):
            # Calculate absorption coefficient for each frequency.
            sea_absorb.append(zf.zplsc_c_absorbtion(temperature, self.params.Pressure, self.params.Salinity,
                                                    profile_hdr.frequency[chan]))

        return sound_speed, depth_range, sea_absorb


In [49]:
from collections import defaultdict


In [50]:
from struct import unpack_from, unpack

In [51]:
import numpy as np
import os
import re

In [52]:
from datetime import datetime as dt

In [53]:
class DataParticleKey():
    PKT_FORMAT_ID = "pkt_format_id"
    PKT_VERSION = "pkt_version"
    STREAM_NAME = "stream_name"
    INTERNAL_TIMESTAMP = "internal_timestamp"
    PORT_TIMESTAMP = "port_timestamp"
    DRIVER_TIMESTAMP = "driver_timestamp"
    PREFERRED_TIMESTAMP = "preferred_timestamp"
    QUALITY_FLAG = "quality_flag"
    VALUES = "values"
    VALUE_ID = "value_id"
    VALUE = "value"
    BINARY = "binary"
    NEW_SEQUENCE = "new_sequence"


In [54]:
class Parser(object):
    """ abstract class to show API needed for plugin poller objects """


        
        

    def get_records(self, max_count):
        """
        Returns a list of particles (following the instrument driver structure).
        """
        raise NotImplementedException("get_records() not overridden!")

    def _publish_sample(self, samples):
        """
        Publish the samples with the given publishing callback.
        @param samples The list of data particle to publish up to the system
        """
        if isinstance(samples, list):
            self._publish_callback(samples)
        else:
            self._publish_callback([samples])

    def _extract_sample(self, particle_class, regex, raw_data, port_timestamp=None, internal_timestamp=None,
                        preferred_ts=DataParticleKey.INTERNAL_TIMESTAMP):
        """
        Extract sample from a response line if present and publish
        parsed particle

        @param particle_class The class to instantiate for this specific
            data particle. Parameterizing this allows for simple, standard
            behavior from this routine
        @param regex The regular expression that matches a data sample if regex
                     is none then process every line
        @param raw_data data to input into this particle.
        @param port_timestamp the port_timestamp (default: None)
        @param internal_timestamp the internal_timestamp (default: None)
        @param preferred_ts the preferred timestamp (default: INTERNAL_TIMESTAMP)
        @retval return a raw particle if a sample was found, else None
        """

        particle = None

        try:
            if regex is None or regex.match(raw_data):
                particle = particle_class(raw_data, port_timestamp=port_timestamp, internal_timestamp=internal_timestamp,
                                          preferred_timestamp=preferred_ts)

                # need to actually parse the particle fields to find out of there are errors
                particle.generate_dict()
                encoding_errors = particle.get_encoding_errors()
                if encoding_errors:
                    log.warn("Failed to encode: %s", encoding_errors)
                    raise SampleEncodingException("Failed to encode: %s" % encoding_errors)

        except (RecoverableSampleException, SampleEncodingException) as e:
            log.error("Sample exception detected: %s raw data: %s", e, raw_data)
            if self._exception_callback:
                self._exception_callback(e)
            else:
                raise e

        return particle



In [55]:
class SimpleParser(Parser):

    def __init__(self, config, stream_handle, exception_callback):
        """
        Initialize the simple parser, which does not use state or the chunker
        and sieve functions.
        @param config: The parser configuration dictionary
        @param stream_handle: The stream handle of the file to parse
        @param exception_callback: The callback to use when an exception occurs
        """

        # the record buffer which will store all parsed particles
        self._record_buffer = []
        # a flag indicating if the file has been parsed or not
        self._file_parsed = False



    def parse_file(self):
        """
        This method must be overridden.  This method should open and read the file and parser the data within, and at
        the end of this method self._record_buffer will be filled with all the particles in the file.
        """
        raise NotImplementedException("parse_file() not overridden!")

    def get_records(self, number_requested=1):
        """
        Initiate parsing the file if it has not been done already, and pop particles off the record buffer to
        return as many as requested if they are available in the buffer.
        @param number_requested the number of records requested to be returned
        @return an array of particles, with a length of the number requested or less
        """
        particles_to_return = []

        if number_requested > 0:
            if self._file_parsed is False:
                self.parse_file()
                self._file_parsed = True

        while len(particles_to_return) < number_requested and len(self._record_buffer) > 0:
            particles_to_return.append(self._record_buffer.pop(0))

        return particles_to_return


In [56]:
PROFILE_DATA_DELIMITER = '\xfd\x02'  # Byte Offset 0 and 1

In [57]:
class ZplscCParticleKey():
    """
    Class that defines fields that need to be extracted for the data particle.
    """
    TRANS_TIMESTAMP = "zplsc_c_transmission_timestamp"
    SERIAL_NUMBER = "serial_number"
    PHASE = "zplsc_c_phase"
    BURST_NUMBER = "burst_number"
    TILT_X = "zplsc_c_tilt_x_counts"
    TILT_Y = "zplsc_c_tilt_y_counts"
    BATTERY_VOLTAGE = "zplsc_c_battery_voltage_counts"
    TEMPERATURE = "zplsc_c_temperature_counts"
    PRESSURE = "zplsc_c_pressure_counts"
    IS_AVERAGED_DATA = "zplsc_c_is_averaged_data"
    FREQ_CHAN_1 = "zplsc_frequency_channel_1"
    VALS_CHAN_1 = "zplsc_values_channel_1"
    DEPTH_CHAN_1 = "zplsc_depth_range_channel_1"
    FREQ_CHAN_2 = "zplsc_frequency_channel_2"
    VALS_CHAN_2 = "zplsc_values_channel_2"
    DEPTH_CHAN_2 = "zplsc_depth_range_channel_2"
    FREQ_CHAN_3 = "zplsc_frequency_channel_3"
    VALS_CHAN_3 = "zplsc_values_channel_3"
    DEPTH_CHAN_3 = "zplsc_depth_range_channel_3"
    FREQ_CHAN_4 = "zplsc_frequency_channel_4"
    VALS_CHAN_4 = "zplsc_values_channel_4"
    DEPTH_CHAN_4 = "zplsc_depth_range_channel_4"

In [58]:
class AzfpProfileHeader():
    _pack_ = 1                              # 124 bytes in the header (includes the 2 byte delimiter)
    _fields_ = [                            # V Byte Offset (from delimiter)
        ('burst_num', 'i2'),            # 002 - Burst number
        ('serial_num', 'i2'),           # 004 - Instrument Serial number
        ('ping_status', 'i2'),          # 006 - Ping Status
        ('burst_interval', 'i2'),         # 008 - Burst Interval (seconds)
        ('year', 'i2'),                 # 012 - Year
        ('month', 'i2'),                # 014 - Month
        ('day', 'i2'),                  # 016 - Day
        ('hour', 'i2'),                 # 018 - Hour
        ('minute', 'i2'),               # 020 - Minute
        ('second', 'i2'),               # 022 - Second
        ('hundredths', 'i2'),           # 024 - Hundreths of a second
        ('digitization_rate', 'i2'*4),  # 026 - Digitization Rate (channels 1-4) (64000, 40000 or 20000)
        ('lockout_index', 'i2'*4),      # 034 - The sample number of samples skipped at start of ping (channels 1-4)
        ('num_bins', 'i2'*4),           # 042 - Number of bins (channels 1-4)
        ('range_samples', 'i2'*4),      # 050 - Range samples per bin (channels 1-4)
        ('num_pings_profile', 'i2'),    # 058 - Number of pings per profile
        ('is_averaged_pings', 'i2'),    # 060 - Indicates if pings are averaged in time
        ('num_pings_burst', 'i2'),      # 062 - Number of pings that have been acquired in this burst
        ('ping_period', 'i2'),          # 064 - Ping period in seconds
        ('first_ping', 'i2'),           # 066 - First ping number (if averaged, first averaged ping number)
        ('second_ping', 'i2'),          # 068 - Last ping number (if averaged, last averaged ping number)
        ('is_averaged_data', 'i2'*4),    # 070 - 1 = averaged data (5 bytes), 0 = not averaged (2 bytes)
        ('error_num', 'i2'),            # 074 - Error number if an error occurred
        ('phase', 'i2'),                 # 076 - Phase used to acquire this profile
        ('is_overrun', 'i2'),            # 077 - 1 if an over run occurred
        ('num_channels', 'i2'),          # 078 - Number of channels (1, 2, 3 or 4)
        ('gain', 'i2'*4),                # 079 - Gain (channels 1-4) 0, 1, 2, 3 (Obsolete)
        ('spare', 'i2'),                 # 083 - Spare
        ('pulse_length', 'i2'*4),       # 084 - Pulse length (channels 1-4) (uS)
        ('board_num', 'i2''i2'*4),          # 092 - Board number of the data (channels 1-4)
        ('frequency', 'i2'*4),          # 100 - Board frequency (channels 1-4)
        ('is_sensor_available', 'i2'),  # 108 - Indicate if pressure/temperature sensor is available
        ('tilt_x', 'i2'),               # 110 - Tilt X (counts)
        ('tilt_y', 'i2'),               # 112 - Tilt Y (counts)
        ('battery_voltage', 'i2'),      # 114 - Battery voltage (counts)
        ('pressure', 'i2'),             # 116 - Pressure (counts)
        ('temperature', 'i2'),          # 118 - Temperature (counts)
        ('ad_channel_6', 'i2'),         # 120 - AD channel 6
        ('ad_channel_7', 'i2')          # 122 - AD channel 7
        ]


In [59]:
def generate_image_file_path(filepath, output_path=None):
    # Extract the file time from the file name
    absolute_path = os.path.abspath(filepath)
    filename = os.path.basename(absolute_path).upper()
    directory_name = os.path.dirname(absolute_path)

    output_path = directory_name if output_path is None else output_path
    image_file = filename.replace('.01A', '.png')
    return os.path.join(output_path, image_file)


In [60]:
class ZplscCCalibrationCoefficients(object):
    # TODO: This class should be replaced by methods to get the CCs from the system.
    DS = list()

    # Freq 38kHz
    DS.append(2.280000038445e-2)

    # Freq 125kHz
    DS.append(2.280000038445e-2)

    # Freq 200kHz
    DS.append(2.250000089407e-2)

    # Freq 455kHz
    DS.append(2.300000004470e-2)


In [61]:
class ZplscCRecoveredDataParticle(DataParticle):
    __metaclass__ = METACLASS

    def __init__(self, *args, **kwargs):
        super(ZplscCRecoveredDataParticle, self).__init__(*args, **kwargs)
        self._data_particle_type = DataParticleType.ZPLSC_C_PARTICLE_TYPE

    def _build_parsed_values(self):
        """
        Build parsed values for Instrument Data Particle.
        @return: list containing type encoded "particle value id:value" dictionary pairs
        """
        # Particle Mapping table, where each entry is a tuple containing the particle
        # field name, count(or count reference) and a function to use for data conversion.

        port_timestamp = self.raw_data[ZplscCParticleKey.TRANS_TIMESTAMP]
        self.contents[DataParticleKey.PORT_TIMESTAMP] = port_timestamp

        return [{DataParticleKey.VALUE_ID: name, DataParticleKey.VALUE: None}
                if self.raw_data[name] is None else
                {DataParticleKey.VALUE_ID: name, DataParticleKey.VALUE: value}
                for name, value in self.raw_data.iteritems()]

NameError: name 'METACLASS' is not defined

In [92]:
class ZplscCParser(SimpleParser):
    
    def __init__(self, config, stream_handle, exception_callback):
        super(ZplscCParser, self).__init__(config, stream_handle, exception_callback)
        self._particle_type = None
        self._gen = None
        self.ph = None  # The profile header of the current record being processed.
        self.cc = ZplscCCalibrationCoefficients()
        self.is_first_record = True
        self.hourly_avg_temp = 0
        self.zplsc_echogram = ZPLSCCEchogram()
        print('Hola from init')
        print(stream_handle)
        self._stream_handle=stream_handle
        print(self._stream_handle)
        
    def find_next_record(self):
        print('Hola from find_next_record')
        good_delimiter = True
        delimiter = self._stream_handle.read(2)
        while delimiter not in [PROFILE_DATA_DELIMITER, '']:
            
            good_delimiter = False
            delimiter = delimiter[1:2]
            delimiter += self._stream_handle.read(1)
        print('Adios while de find_next_record')
        if not good_delimiter:
            self._exception_callback('Invalid record delimiter found.\n')
        print('Adios find_next_record')
    def parse_record(self):
        """
        Parse one profile data record of the zplsc-c data file.
        """
        chan_values = [[], [], [], []]
        overflow_values = [[], [], [], []]

        # Parse the data values portion of the record.
        for chan in range(self.ph.num_channels):
            num_bins = self.ph.num_bins[chan]

            # Set the data structure format for the scientific data, based on whether
            # the data is averaged or not. Construct the data structure and read the
            # data bytes for the current channel. Unpack the data based on the structure.
            if self.ph.is_averaged_data[chan]:
                data_struct_format = '>' + str(num_bins) + 'I'
            else:
                data_struct_format = '>' + str(num_bins) + 'H'
            data_struct = struct.Struct(data_struct_format)
            data = self._stream_handle.read(data_struct.size)
            chan_values[chan] = data_struct.unpack(data)

            # If the data type is for averaged data, calculate the averaged data taking the
            # the linear sum channel values and overflow values and using calculations from
            # ASL MatLab code.
            if self.ph.is_averaged_data[chan]:
                overflow_struct_format = '>' + str(num_bins) + 'B'
                overflow_struct = struct.Struct(overflow_struct_format)
                overflow_data = self._stream_handle.read(num_bins)
                overflow_values[chan] = overflow_struct.unpack(overflow_data)

                if self.ph.is_averaged_pings:
                    divisor = self.ph.num_pings_profile * self.ph.range_samples[chan]
                else:
                    divisor = self.ph.range_samples[chan]

                linear_sum_values = np.array(chan_values[chan])
                linear_overflow_values = np.array(overflow_values[chan])

                values = (linear_sum_values + (linear_overflow_values * 0xFFFFFFFF))/divisor
                values = (np.log10(values) - 2.5) * (8*0xFFFF) * self.cc.DS[chan]
                values[np.isinf(values)] = 0
                chan_values[chan] = values

        # Convert the date and time parameters to a epoch time from 01-01-1900.
        timestamp = (datetime(self.ph.year, self.ph.month, self.ph.day,
                              self.ph.hour, self.ph.minute, self.ph.second,
                              (self.ph.hundredths * 10000)) - datetime(1900, 1, 1)).total_seconds()

        sound_speed, depth_range, sea_absorb = self.zplsc_echogram.compute_echogram_metadata(self.ph)

        chan_values = self.zplsc_echogram.compute_backscatter(self.ph, chan_values, sound_speed, depth_range,
                                                              sea_absorb)

        zplsc_particle_data = {
            ZplscCParticleKey.TRANS_TIMESTAMP: timestamp,
            ZplscCParticleKey.SERIAL_NUMBER: str(self.ph.serial_num),
            ZplscCParticleKey.PHASE: self.ph.phase,
            ZplscCParticleKey.BURST_NUMBER: self.ph.burst_num,
            ZplscCParticleKey.TILT_X: self.ph.tilt_x,
            ZplscCParticleKey.TILT_Y: self.ph.tilt_y,
            ZplscCParticleKey.BATTERY_VOLTAGE: self.ph.battery_voltage,
            ZplscCParticleKey.PRESSURE: self.ph.pressure,
            ZplscCParticleKey.TEMPERATURE: self.ph.temperature,
            ZplscCParticleKey.IS_AVERAGED_DATA: list(self.ph.is_averaged_data),
            ZplscCParticleKey.FREQ_CHAN_1: float(self.ph.frequency[0]),
            ZplscCParticleKey.VALS_CHAN_1: list(chan_values[0]),
            ZplscCParticleKey.DEPTH_CHAN_1: list(depth_range[0]),
            ZplscCParticleKey.FREQ_CHAN_2: float(self.ph.frequency[1]),
            ZplscCParticleKey.VALS_CHAN_2: list(chan_values[1]),
            ZplscCParticleKey.DEPTH_CHAN_2: list(depth_range[1]),
            ZplscCParticleKey.FREQ_CHAN_3: float(self.ph.frequency[2]),
            ZplscCParticleKey.VALS_CHAN_3: list(chan_values[2]),
            ZplscCParticleKey.DEPTH_CHAN_3: list(depth_range[2]),
            ZplscCParticleKey.FREQ_CHAN_4: float(self.ph.frequency[3]),
            ZplscCParticleKey.VALS_CHAN_4: list(chan_values[3]),
            ZplscCParticleKey.DEPTH_CHAN_4: list(depth_range[3])
        }

        return zplsc_particle_data, timestamp, chan_values, depth_range

    def parse_file(self):
        self.ph = AzfpProfileHeader()
        self.find_next_record()
        while self._stream_handle.readinto(self.ph):
            try:
                # Parse the current record
                zplsc_particle_data, timestamp, _, _ = self.parse_record()

                # Create the data particle
                particle = self._extract_sample(ZplscCRecoveredDataParticle, None, zplsc_particle_data, timestamp,
                                                timestamp, DataParticleKey.PORT_TIMESTAMP)
                if particle is not None:
                    log.trace('Parsed particle: %s' % particle.generate_dict())
                    self._record_buffer.append(particle)

            except (IOError, OSError) as ex:
                self._exception_callback('Reading stream handle: %s: %s\n' % (self._stream_handle.name, ex.message))
                return
            except struct.error as ex:
                self._exception_callback('Unpacking the data from the data structure: %s' % ex.message)
            except exceptions.ValueError as ex:
                self._exception_callback('Transition timestamp has invalid format: %s' % ex.message)
            except (SampleException, RecoverableSampleException) as ex:
                self._exception_callback('Creating data particle: %s' % ex.message)

            # Clear the profile header data structure and find the next record.
            self.ph = AzfpProfileHeader()
            self.find_next_record()
            

    def create_echogram(self, echogram_file_path=None):
        """
        Parse the *.O1A zplsc_c data file and create the echogram from this data.
        :param echogram_file_path: Path to store the echogram locally.
        :return:
        """
        
        import logging
        sv_dict = {}
        data_times = []
        frequencies = {}
        depth_range = []
        
        print('Hola from create echogram')
        
        input_file_path ='18030100.01A'#self._stream_handle.name
        logging.info('Begin processing echogram data: %r', input_file_path)
        image_path = generate_image_file_path(input_file_path, echogram_file_path)

        self.ph = AzfpProfileHeader()
        self.find_next_record()
        while self._stream_handle.readinto(self.ph):
            try:
                _, timestamp, chan_data, depth_range = self.parse_record()

                if not sv_dict:
                    range_chan_data = range(1, len(chan_data)+1)
                    sv_dict = {channel: [] for channel in range_chan_data}
                    frequencies = {channel: float(self.ph.frequency[channel-1]) for channel in range_chan_data}
                    print('Hola desde while de create echogram1')
                for channel in sv_dict:
                    sv_dict[channel].append(chan_data[channel-1])
                    print('Hola desde while de create echogram2')
                data_times.append(timestamp)

            except (IOError, OSError) as ex:
                self._exception_callback(ex)
                return
            except struct.error as ex:
                self._exception_callback(ex)
            except exceptions.ValueError as ex:
                self._exception_callback(ex)
            except (SampleException, RecoverableSampleException) as ex:
                self._exception_callback(ex)

            # Clear the profile header data structure and find the next record.
            self.ph = AzfpProfileHeader()
            self.find_next_record()

        logging.info('Completed processing all data: %r', input_file_path)

        data_times = np.array(data_times)

        for channel in sv_dict:
            sv_dict[channel] = np.array(sv_dict[channel])

        logging.info('Begin generating echogram: %r', image_path)

        plot = ZPLSPlot(data_times, sv_dict, frequencies, depth_range[0][-1], depth_range[0][0])
        plot.generate_plots()
        plot.write_image(image_path)

        log.info('Completed generating echogram: %r', image_path)

In [93]:
class ParticleDataHandler(object):
    def __init__(self):particle_data_handler
    def addParticleSample(self, sample_type, sample):
        log.debug("Sample type: %s, Sample data: %s", sample_type, sample)
        self._samples.setdefault(sample_type, []).append(sample)

    def setParticleDataCaptureFailure(self):
        log.debug("Particle data capture failed")
        self._failure = True
        

In [94]:
def rec_exception_callback(exception):
        """
        Callback function to log exceptions and continue.

        @param exception - Exception that occurred
        """

        log.info("Exception occurred: %s", exception.message)


In [95]:
class DataSetDriverConfigKeys():
    PARTICLE_MODULE = "particle_module"
    PARTICLE_CLASS = "particle_class"
    PARTICLE_CLASSES_DICT = "particle_classes_dict"
    DIRECTORY = "directory"
    STORAGE_DIRECTORY = "storage_directory"
    PATTERN = "pattern"
    FREQUENCY = "frequency"
    FILE_MOD_WAIT_TIME = "file_mod_wait_time"
    HARVESTER = "harvester"
    PARSER = "parser"
    MODULE = "module"
    CLASS = "class"
    URI = "uri"
    CLASS_ARGS = "class_args"


In [96]:
print('prueba')


prueba


In [97]:
input_file_path = '18030100.01A'
MODULE_NAME = 'mi.dataset.parser.zplsc_c'
CLASS_NAME = 'ZplscCRecoveredDataParticle'
CONFIG = {
    DataSetDriverConfigKeys.PARTICLE_MODULE: MODULE_NAME,
    DataSetDriverConfigKeys.PARTICLE_CLASS: CLASS_NAME
}
#Nor sure if the first ZplscCParser argument should be CONFIG or None.
zplsc_echogram_file_path = 'C:\Oceanhackweek\proyecto\AFZP_matlab'
parser = ZplscCParser(None,  open(input_file_path, 'rb'), rec_exception_callback)
zplsc_echogram_file_path = 'C:\Oceanhackweek\proyecto\AFZP_matlab'
parser.create_echogram(zplsc_echogram_file_path) 



Hola from init
<_io.BufferedReader name='18030100.01A'>
<_io.BufferedReader name='18030100.01A'>
Hola from create echogram
Hola from find_next_record


KeyboardInterrupt: 

In [70]:
def compute_sv_offset(frequency, pulse_length):
    """
    A correction must be made to compensate for the effects of the finite response
    times of both the receiving and transmitting parts of the instrument. The magnitude
    of the correction will depend on the length of the transmitted pulse, and the response
    time (on both transmission and reception) of the instrument.

    :param frequency: Frequency in KHz
    :param pulse_length: Pulse length in uSecs
    :return:
    """

    sv_offset = 0

    if frequency > 38:  # 125,200,455,769 kHz
        if pulse_length == 300:
            sv_offset = 1.1
        elif pulse_length == 500:
            sv_offset = 0.8
        elif pulse_length == 700:
            sv_offset = 0.5
        elif pulse_length == 900:
            sv_offset = 0.3
        elif pulse_length == 1000:
            sv_offset = 0.3
    else:  # 38 kHz
        if pulse_length == 500:
            sv_offset = 1.1
        elif pulse_length == 1000:
            sv_offset = 0.7

    return sv_offset


In [71]:
# de fichero  C:\Oceanhackweek\proyecto\AFZP_matlab\mi_instrument\mi\dataset\driver\zplsc_c\zplsc_functions.py
decompress_power = np.array(power) * 10. * np.log10(2) / 256.
vin = 2.5 * (counts / 65535)
r = (ka + kb*vin) / (kc - vin)
temperature = 1 / (a + b * (np.log(r)) + c * (np.log(r)**3)) - 273
tilt = a + (b * counts) + (c * counts**2) + (d * counts**3)
z = t/10
sea_c = 1449.05 + (z * (45.7 + z*((-5.21) + 0.23*z))) + ((1.333 + z*((-0.126) + z*0.009)) * (s-35.0)) + \(p/1000)*(16.3+0.18*(p/1000))
# Calculate relaxation frequencies
t_k = t + 273.0
f1 = 1320.0*t_k * np.exp(-1700/t_k)
f2 = 1.55e7*t_k * np.exp(-3052/t_k)

# Coefficients for absorption equations
k = 1 + p/10.0
a = 8.95e-8 * (1 + t*(2.29e-2 - 5.08e-4*t))
b = (s/35.0)*4.88e-7*(1+0.0134*t)*(1-0.00103*k + 3.7e-7*(k*k))
c = 4.86e-13*(1+t*((-0.042)+t*(8.53e-4-t*6.23e-6)))*(1+k*(-3.84e-4+k*7.57e-8))
freqk = freq*1000
sea_abs = (a*f1*(freqk**2))/((f1*f1)+(freqk**2))+(b*f2*(freqk**2))/((f2*f2)+(freqk**2))+c*(freqk**2)

#aplicar compute_sv_offset(frequency, pulse_length)

SyntaxError: unexpected character after line continuation character (<ipython-input-71-aa1b93d6ffc6>, line 8)

In [72]:
# Set contants for unpacking .raw files
#BLOCK_SIZE = 1024*4             # Block size read in from binary file to search for token
#LENGTH_SIZE = 4
 #   byte_cnt = LENGTH_SIZE
#
    # Configuration datagram header
 #   byte_cnt += DATAGRAM_HEADER_SIZE
#
    # Configuration: header
  #  config_header = read_config_header(raw[byte_cnt:byte_cnt+CONFIG_HEADER_SIZE])
   # byte_cnt += CONFIG_HEADER_SIZE
   #config_transducer = []
    #for num_transducer in range(config_header['transducer_count']):
     #   config_transducer.append(read_config_transducer(raw[byte_cnt:byte_cnt+CONFIG_TRANSDUCER_SIZE]))
      #  byte_cnt += CONFIG_TRANSDUCER_SIZE
   

    #raw = filehandle.read(BLOCK_SIZE)




In [73]:
import matplotlib.pyplot as plt

In [74]:
plt.imshow(power_data_dict[38000.0],aspect='auto')
plt.colorbar()
plt.show()

NameError: name 'power_data_dict' is not defined

In [ ]:
plt.imshow(power_data_dict[120000.0],aspect='auto')
plt.colorbar()
plt.show()